In [20]:
import pandas as pd
import uuid

In [21]:
df_product = pd.read_csv('../data/Products_with_Prices.csv')
df_transaction = pd.read_csv('../data/Transactions.csv')

In [22]:
df_product

,productId,productName,price
0,1,tropical fruit,7.803532
1,2,whole milk,1.800000
2,3,pip fruit,3.000000
3,4,other vegetables,0.800000
4,5,rolls/buns,1.200000
...,...,...,...
162,163,pudding powder,4.900000
163,164,ready soups,3.900000
164,165,make up remover,3.400000
165,166,toilet cleaner,4.900000


In [23]:
df_transaction

,Member_number,Date,productId,items
0,1808,21-07-2015,1,3
1,2552,05-01-2015,2,1
2,2300,19-09-2015,3,3
3,1187,12-12-2015,4,3
4,3037,01-02-2015,2,1
...,...,...,...,...
38760,4471,08-10-2014,76,2
38761,2022,23-02-2014,65,3
38762,1097,16-04-2014,154,2
38763,1510,03-12-2014,12,3


In [24]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Member_number  38765 non-null  int64 
 1   Date           38765 non-null  object
 2   productId      38765 non-null  int64 
 3   items          38765 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.2+ MB


In [25]:
# Check for product null values
df_product.isnull().sum()

productId      0
productName    0
price          0
dtype: int64

In [26]:
# Check for transaction null values
df_transaction.isnull().sum()

Member_number    0
Date             0
productId        0
items            0
dtype: int64

In [27]:
# Check for unique values in product
df_product.nunique()

productId      167
productName    167
price           91
dtype: int64

In [28]:
# Check for unique values in transaction
df_transaction.nunique()

Member_number    3898
Date              728
productId         167
items               3
dtype: int64

In [29]:
# A vertical lookup for the product and add its price, name to the transaction.
df_transaction = df_transaction.merge(df_product[['productId', 'productName', 'price']], on='productId', how='left')

In [30]:
# Add uuid to ID the transaction
df_transaction['TransactionID'] = [str(uuid.uuid4()) for _ in range(len(df_transaction))]

In [31]:
df_transaction.head()

,Member_number,Date,productId,items,productName,price,TransactionID
0,1808,21-07-2015,1,3,tropical fruit,7.803532,9b492ede-146e-461e-8fe3-85fb13ad28c3
1,2552,05-01-2015,2,1,whole milk,1.800000,0e3b83a1-0b57-417a-854c-ae542d1d2ea5
2,2300,19-09-2015,3,3,pip fruit,3.000000,9fc5bb3c-28c2-4fe3-b1c0-5e4965eb1a3e
3,1187,12-12-2015,4,3,other vegetables,0.800000,1df0af3c-111b-4b45-b308-303b1d6f71d6
4,3037,01-02-2015,2,1,whole milk,1.800000,dbe3239f-5028-4b03-8088-0ee7c0e7a833


In [32]:
# Convert the date columns to datetime
if 'Date' in df_transaction.columns:
    df_transaction['TransactionDate'] = pd.to_datetime(df_transaction['Date'], dayfirst=True)
    df_transaction.drop('Date', axis=1, inplace=True)

In [33]:
# Calculate the total payment of the transaction
df_transaction['TotalPayment'] = df_transaction['price'] * df_transaction['items']

In [34]:
df_transaction.head()

,Member_number,productId,items,productName,price,TransactionID,TransactionDate,TotalPayment
0,1808,1,3,tropical fruit,7.803532,9b492ede-146e-461e-8fe3-85fb13ad28c3,2015-07-21,23.410597
1,2552,2,1,whole milk,1.800000,0e3b83a1-0b57-417a-854c-ae542d1d2ea5,2015-01-05,1.800000
2,2300,3,3,pip fruit,3.000000,9fc5bb3c-28c2-4fe3-b1c0-5e4965eb1a3e,2015-09-19,9.000000
3,1187,4,3,other vegetables,0.800000,1df0af3c-111b-4b45-b308-303b1d6f71d6,2015-12-12,2.400000
4,3037,2,1,whole milk,1.800000,dbe3239f-5028-4b03-8088-0ee7c0e7a833,2015-02-01,1.800000


In [35]:
latest_date = df_transaction['TransactionDate'].max()
latest_date

Timestamp('2015-12-30 00:00:00')

In [36]:
# All the column so far
df_transaction.columns.to_list()

['Member_number',
 'productId',
 'items',
 'productName',
 'price',
 'TransactionID',
 'TransactionDate',
 'TotalPayment']

In [40]:
recency = lambda x: (latest_date - x.max()).days
frequency = lambda x: len(x.unique())
monetary = lambda x: round(x.sum(), 2)

df_rfm = df_transaction.groupby('Member_number').agg(
    Recency=('TransactionDate', recency),
    Frequency=('TransactionID', frequency),
    Monetary=('TotalPayment', monetary)
)

In [41]:
df_rfm.columns = ['Recency', 'Frequency', 'Monetary']
df_rfm.sort_values('Monetary', ascending=False, inplace=True)

In [42]:
df_rfm

,Recency,Frequency,Monetary
Member_number,,,
2433,3,31,375.31
2193,91,27,361.45
1793,26,25,345.10
3289,4,29,334.17
2743,142,17,312.47
...,...,...,...
4565,0,2,2.20
3949,273,2,2.10
1560,593,2,1.90


In [43]:
df_rfm.describe()

,Recency,Frequency,Monetary
count,3898.000000,3898.000000,3898.000000
mean,187.701385,9.944844,85.214623
std,159.912688,5.310796,55.621118
min,0.000000,2.000000,1.300000
25%,57.000000,6.000000,43.810000
50%,141.000000,9.000000,74.900000
75%,280.000000,13.000000,115.295000
max,727.000000,36.000000,375.310000
